# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("/Users/matthewflanagan/Desktop/Data Analytics Work/Module 6/HW/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,turpan,42.9333,89.1667,27.01,57,12,1.68,CN,2024-01-04 21:45:34
1,1,waitangi,-43.9535,-176.5597,62.82,88,100,26.64,NZ,2024-01-04 21:45:35
2,2,berekum,7.4534,-2.5840,82.47,50,99,6.11,GH,2024-01-04 21:45:36
3,3,iqaluit,63.7506,-68.5145,2.93,60,75,3.44,CA,2024-01-04 21:45:36
4,4,yellowknife,62.4560,-114.3525,-28.62,83,75,5.75,CA,2024-01-04 21:45:36


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 500,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
temp_criteria = (city_data_df['Max Temp (F)'] > 60) & (city_data_df['Max Temp (F)'] < 105)
humidity_criteria = city_data_df['Humidity'] < 70
cloud_criteria = city_data_df['Cloudiness'] < 25
wind_criteria = city_data_df['Wind Speed'] < 5

ideal_cities = city_data_df[temp_criteria & humidity_criteria & cloud_criteria & wind_criteria]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities

,City_ID,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
104,104,kyabe,9.4515,18.9449,78.30,15,5,3.51,TD,2024-01-04 21:46:00
127,127,salalah,17.0151,54.0924,73.49,68,0,4.61,OM,2024-01-04 21:46:07
158,158,kone,-21.0595,164.8658,87.51,54,1,1.21,NC,2024-01-04 21:46:26
197,197,sakti,22.0333,82.9667,62.94,55,10,4.25,IN,2024-01-04 21:46:42
225,225,rumbek,6.8020,29.6912,78.75,19,0,4.63,SS,2024-01-04 21:46:50
247,247,al ghayzah,16.2079,52.1760,73.26,62,9,1.14,YE,2024-01-04 21:46:56
261,261,shantipur,23.2500,88.4667,61.30,42,0,4.63,IN,2024-01-04 21:46:59
354,354,sidi ifni,29.3797,-10.1730,64.81,55,0,4.56,MA,2024-01-04 21:47:24
368,368,odienne,9.5100,-7.5692,79.29,22,8,1.79,CI,2024-01-04 21:38:25


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
104,kyabe,TD,9.4515,18.9449,15,
127,salalah,OM,17.0151,54.0924,68,
158,kone,NC,-21.0595,164.8658,54,
197,sakti,IN,22.0333,82.9667,55,
225,rumbek,SS,6.8020,29.6912,19,
247,al ghayzah,YE,16.2079,52.1760,62,
261,shantipur,IN,23.2500,88.4667,42,
354,sidi ifni,MA,29.3797,-10.1730,55,
368,odienne,CI,9.5100,-7.5692,22,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "Radius": radius,
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format

    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kyabe - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
kone - nearest hotel: Pacifik Appartels
sakti - nearest hotel: No hotel found
rumbek - nearest hotel: Hillview Hotel
al ghayzah - nearest hotel: فندق تاج العرب
shantipur - nearest hotel: No hotel found
sidi ifni - nearest hotel: Hotel Ére Nouvelle
odienne - nearest hotel: Hotel Frontière


,City,Country,Lat,Lng,Humidity,Hotel Name
104,kyabe,TD,9.4515,18.9449,15,No hotel found
127,salalah,OM,17.0151,54.0924,68,Muscat International Hotel
158,kone,NC,-21.0595,164.8658,54,Pacifik Appartels
197,sakti,IN,22.0333,82.9667,55,No hotel found
225,rumbek,SS,6.8020,29.6912,19,Hillview Hotel
247,al ghayzah,YE,16.2079,52.1760,62,فندق تاج العرب
261,shantipur,IN,23.2500,88.4667,42,No hotel found
354,sidi ifni,MA,29.3797,-10.1730,55,Hotel Ére Nouvelle
368,odienne,CI,9.5100,-7.5692,22,Hotel Frontière


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 400,
    frame_height = 200,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)